```
Optuna란 하이퍼파라미터 최적화 태스크를 자동화해주는 프레임워크로, 다음과 같은 장점이 있다.

거의 모든 ML/DL 프레임워크에서 사용 가능한 넓은 범용성을 가지고 있다.
간단하고 빠르다.
최신 동향의 다양한 최적화 알고리즘을 갖추고 있다.
병렬 처리가 가능하다.
간단한 메소드로 시각화가 가능

Optuna를 이해하기 위해서는 다음의 용어에 익숙해져야 한다.
Study: 목적 함수에 기반한 최적화
Trial: 목적함수 시행
쉽게 말해 study는 최적화를 하는 과정이고, trial은 다양한 조합으로 목적함수를 시행하는 횟수를 뜻한다.
Study의 목적은 여러 번의 trial을 거쳐 최적의 하이퍼파라미터 조합을 찾는 것이라고 할 수 있겠다.
```

![nn](optuna.png)
## 복사는 밑에 코드블럭에서

In [ ]:
# Integer parameter : A uniform distribution on integers.
# n_estimators = trial.suggest_int('n_estimators',100,500)

# Categorical parameter : A categorical distribution.
# criterion = trial.suggest_categorical('criterion' ,['gini', 'entropy'])

# Uniform parameter : A uniform distribution in linear domain.
# subsample = trial.suggest_uniform('subsample' ,0.2,0.8)

# Discrete-uniform parameter : A discretized uniform distribution in linear domain.
# max_features = trial.suggest_discrete_uniform('max_features', 0.05,1,0.05)

# Loguniform parameter : A uniform distribution in log domain. ** learning_rate = trial.sugget_loguniform('learning_rate' : 1e-6, 1e-3)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, precision_score

In [ ]:
X= 
y=  

In [ ]:
X_train,X_test,y_train,y_test= train_test_split(X, y, test_size=0.3, random_state=156) 

In [ ]:
import optuna

# 1. 최소화/최대화할 목적함수 정의
def objective(trial):
    iris = sklearn.datasets.load_iris()
    x, y = iris.data, iris.target

# 2. trial object로 하이퍼파라미터 값 추천
# 다양한 분류모델을 설정해서 비교할 수 있다.
    classifier_name = trial.suggest_categorical('classifier', ['SVC', 'LogisticRegression','DecisionTreeClassifier','GradientBoostingClassifier','KNeighborsClassifier','LGBMClassifier','RandomForest'])
    #분류 모델이 SVC일 때
    if classifier_name == 'SVC':
        svc_c = trial.suggest_loguniform('svc_c', 1e-10, 1e10)
        classifier_obj = svm.SVC(C=svc_c, gamma='auto')
    
    #LogisticRegression
    elif classifier_name == 'LogisticRegression':       
        penalty = trial.suggest_categorical('penalty', ['l1', 'l2', 'elasticnet'])     # l1 is Lasso, l2 is Ridge
        C = trial.suggest_uniform('C', 0.00002,1)
        classifier_obj =  LogisticRegression(penalty=penalty, C=C, solver='liblinear')
    
    #DecisionTreeClassifier
    elif classifier_name == 'DecisionTreeClassifier':       
        max_depth = trial.suggest_int('max_depth',2,20)     
        min_samples_leaf = trial.suggest_discrete_uniform('min_samples_leaf',5,100,5)
        criterion = trial.suggest_categorical('criterion', ["gini", "entropy"]) 
    
        classifier_obj =  DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=min_samples_leaf, criterion=criterion)
    
    #GradientBoostingClassifier
    elif classifier_name == 'GradientBoostingClassifier':
        max_depth = trial.suggest_int('max_depth',2,20)
        learning_rate = trial.suggest_discrete_uniform('learning_rate',0.01,0.2,0.005)
        min_samples_split = trial.suggest_discrete_uniform('min_samples_split',0.1,0.5,0.05)
        min_samples_leaf = trial.suggest_discrete_uniform('min_samples_leaf',0.1,0.5,0.05)
        max_features = trial.suggest_categorical('max_features', ["log2","sqrt"])
        criterion = trial.suggest_categorical('criterion', ["friedman_mse",  "mae"]) #어 얘는 손실함수라 밑에 따로해야할수도.. 
        subsample = trial.suggest_categorical('subsample', [0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0])
        
        classifier_obj =  GradientBoostingClassifier(loss='deviance',n_estimators=10, max_depth=max_depth, learning_rate=learning_rate,min_samples_split=min_samples_split,
                                                    min_samples_leaf=min_samples_leaf,max_features=max_features,criterion=criterion,subsample=subsample)
        
    #KNeighborsClassifier    
    elif classifier_name == 'KNeighborsClassifier':
        n_neighbors = trial.suggest_discrete_uniform('n_neighbors',1,20,1)
        weights = trial.suggest_categorical('weights', ["uniform", "distance"])
        metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 'minkowski'])
        
        classifier_obj =  KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, metric=metric)
    
    #LGBMClassifier    
    elif classifier_name == 'LGBMClassifier':
        
        num_leaves = trial.suggest_discrete_uniform('num_leaves',20,100,10)
        min_child_samples = trial.suggest_discrete_uniform('min_child_samples',5,20,5)
        max_depth = trial.suggest_categorical('max_depth', [-1,5,10,20])
        learning_rate = trial.suggest_discrete_uniform('learning_rate',0.05,0.5,0.05)
        reg_alpha = trial.suggest_discrete_uniform('reg_alpha',0,0.05,0.01)
        
        classifier_obj =  LGBMClassifier(num_leaves=num_leaves,min_child_samples=min_child_samples,max_depth=max_depth,learning_rate=learning_rate,reg_alpha=reg_alpha)
    
    #분류모델이 랜덤포레스트일 때
    else:
        rf_max_depth = int(trial.suggest_loguniform('rf_max_depth', 2, 32))
        classifier_obj = RandomForestClassifier(max_depth=rf_max_depth, n_estimators=10)
    
    accuracy = cross_val_score(classifier_obj,X_train, y_train, cv = 4).mean()
    return accuracy

# 3. study 오브젝트 생성하고 목적함수 최적화하는 단계
# 여기서는 목적함수를 정확도로 설정했기 때문에 최대화를 목표로 하고 있지만, 손실함수의 경우 direction='minimize'로 설정
study = optuna.create_study(direction='maximize')
# 반복 시행 횟수(trial)는 200번으로
study.optimize(objective, n_trials=200)

In [ ]:
# 시행된 trial 중 최적의 하이퍼파라미터 반환하는 메소드
print(study.best_trial.params)

# 시행된 trial 중 가장 높은 값 반환하는 메소드
optuna_acc = study.best_trial.value
print(optuna_acc)